## **Resources**
1. https://www.datacamp.com/community/tutorials/recommender-systems-python/
2. https://blog.codecentric.de/en/2019/07/recommender-system-movie-lens-dataset/
3. http://surpriselib.com/
4. https://towardsdatascience.com/building-and-testing-recommender-systems-with-surprise-step-by-step-d4ba702ef80b
5. https://github.com/NicolasHug/Surprise/blob/master/examples/benchmark.py


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# 1.   Convert Json-File into CSV File

---








In [0]:
import pandas as pd
from ast import literal_eval

with open("/content/drive/My Drive/Web Mining Project/Dataset/steam_reviews.json") as f:
	lines = f.readlines()
	print("done reading")
	dicts = []
	count = 1
	for line in lines:
		if (count%100000 == 0):
			print(count)
		dicts.append(literal_eval(line))
		count=count+1
	df = pd.DataFrame(dicts)
	df.to_csv("/content/drive/My Drive/Web Mining Project/Dataset/steam_reviews.csv")

In [0]:
import pandas as pd
from ast import literal_eval

with open("/content/drive/My Drive/Web Mining Project/Dataset/steam_games.json") as f:
	lines = f.readlines()
	print("done reading")
	dicts = []
	count = 1
	for line in lines:
		if (count%100000 == 0):
			print(count)
		dicts.append(literal_eval(line))
		count=count+1
	df = pd.DataFrame(dicts)
	df.to_csv("/content/drive/My Drive/Web Mining Project/Dataset/steam_games.csv")

done reading


# 2.   Preprocessing Steps





**`Steam_Games`** Rename the column and drop unnecessary columns

In [0]:
import pandas as pd
import numpy as np
df_games = pd.read_csv('/content/drive/My Drive/Web Mining Project/Dataset/steam_games.csv')
#df.dtypes
df_games.rename(columns={'id':'product_id'}, inplace=True)
#drop unnecessary attributes
df_games.drop(['app_name','url', 'reviews_url', 'release_date'], axis=1, inplace=True)
df_games.head()

,Unnamed: 0,publisher,genres,title,tags,discount_price,specs,price,early_access,product_id,developer,sentiment,metascore
0,0,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",4.49,['Single-player'],4.99,False,761140.0,Kotoshiro,NaN,NaN
1,1,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",NaN,"['Single-player', 'Multi-player', 'Online Mult...",Free To Play,False,643980.0,Secret Level SRL,Mostly Positive,NaN
2,2,Poolians.com,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,"['Free to Play', 'Simulation', 'Sports', 'Casu...",NaN,"['Single-player', 'Multi-player', 'Online Mult...",Free to Play,False,670290.0,Poolians.com,Mostly Positive,NaN
3,3,彼岸领域,"['Action', 'Adventure', 'Casual']",弹炸人2222,"['Action', 'Adventure', 'Casual']",0.83,['Single-player'],0.99,False,767400.0,彼岸领域,NaN,NaN
4,4,NaN,NaN,NaN,"['Action', 'Indie', 'Casual', 'Sports']",1.79,"['Single-player', 'Full controller support', '...",2.99,False,773570.0,NaN,NaN,NaN


Steam_Reviews Rename the column and drop the unnecessary columns

In [0]:
df_reviews = pd.read_csv('/content/drive/My Drive/Web Mining Project/Dataset/steam_reviews.csv', keep_default_na=False)
df_reviews.head()
# put text column to another csv
df_text = df_reviews[['user_id','text','date','product_id', 'username']].copy()
df_text.to_csv("/content/drive/My Drive/Web Mining Project/Dataset/user_text.csv", index=False)

df_reviews.drop(['page_order', 'early_access', 'username', 'compensation', 'text', 'page'], axis=1, inplace=True)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
#replace empty places with nan and delete rows where user_id is missing
df_reviews.replace('', np.nan, inplace=True)
df_reviews = df_reviews.dropna(axis=0, subset=['user_id'])
#df2['user_id'] = df2['user_id'].astype(str)
df_reviews.head()

,Unnamed: 0,hours,products,product_id,date,found_funny,user_id
4,4,16.6,577.0,35140,2018-01-04,NaN,76561198007483075
8,8,1.1,729.0,707610,2017-10-16,NaN,76561197970402776
11,11,3.8,431.0,328100,2017-06-23,2.0,76561198060686749
13,13,14.8,147.0,35140,2018-01-03,NaN,76561198023491401
16,16,17.0,87.0,35140,2018-01-03,NaN,76561198115331805


In [0]:
#inner join games and reviews on product_id
merged = df_games.merge(df_reviews, how='inner', on='product_id')
merged = merged.dropna(axis=0, subset=['user_id', 'title'])



In [0]:
merged.dtypes
merged.head()
merged = merged.loc[:, ~merged.columns.str.contains('^Unnamed')]

print("DF size before dropping: " + str(merged.shape))
merged.drop_duplicates(subset=['user_id','title','hours','date'], keep='first', inplace=True)
print("DF size after dropping: " + str(merged.shape))
#merged.to_csv('/content/drive/My Drive/Web Mining Project/Dataset/final_merged.csv', index=False)
#merged.head()

In [0]:
df_usergamehours = merged[['user_id','product_id','title', 'hours']].copy()
df_usergamehours['count']=1
df_usergamehours['hours'] = df_usergamehours['hours'].astype(float)
df_usergamehours.dtypes

#df_grouped = df_usergamehours.groupby(['user_id', 'title']).sum()
#print(df_grouped.shape)
df_grouped = df_usergamehours.groupby(['user_id', 'title', 'product_id']).sum()
print(df_grouped.shape)
df_grouped = df_grouped.rename(columns={'hours':'total_hours'})
print(df_grouped.head())

(2832522, 2)
                                                                       total_hours  count
user_id           title                                    product_id                    
76561197960265806 Terraria                                 105600.0          280.7      1
76561197960266010 Animation Throwdown: The Quest for Cards 591960.0           28.4      1
76561197960266146 South Park™: The Fractured But Whole™    488790.0            2.1      1
76561197960266225 Paladins®                                444090.0           59.2      1
76561197960266375 Fallout 4                                377160.0         1487.0      1


In [0]:
df_grouped.head()
df_grouped.shape

(2832522, 2)

In [0]:
df_grouped=df_grouped.reset_index()
df_grouped.dtypes
df_grouped.head()
#df5000=df_grouped.head(5000).copy()

,user_id,title,product_id,total_hours,count
0,76561197960265806,Terraria,105600.0,280.7,1
1,76561197960266010,Animation Throwdown: The Quest for Cards,591960.0,28.4,1
2,76561197960266146,South Park™: The Fractured But Whole™,488790.0,2.1,1
3,76561197960266225,Paladins®,444090.0,59.2,1
4,76561197960266375,Fallout 4,377160.0,1487.0,1


# 3. Creating Binnings for the individual users



In [0]:
import numpy as np
#df_grouped.dtypes
df_grouped['user_id'] = df_grouped['user_id'].astype('int64')
df_grouped.dtypes
df_grouped['product_id'] = df_grouped['product_id'].astype('int64')
#df_grouped.drop(['level_0', 'index'], axis=1, inplace=True)
df_grouped.head()
df_grouped.to_csv('/content/drive/My Drive/Web Mining Project/Dataset/user_game_totalhours.csv', index=False)

In [0]:
df_grouped.sort_values(['user_id'], inplace = True)
df_grouped.head()

,user_id,title,product_id,total_hours,count
0,76561197960265806,Terraria,105600,280.7,1
1,76561197960266010,Animation Throwdown: The Quest for Cards,591960,28.4,1
2,76561197960266146,South Park™: The Fractured But Whole™,488790,2.1,1
3,76561197960266225,Paladins®,444090,59.2,1
4,76561197960266375,Fallout 4,377160,1487.0,1


In [0]:
df_grouped.shape

(2832522, 5)

In [0]:
import pandas as pd
import numpy as np
df_grouped = pd.read_csv('/content/drive/My Drive/Web Mining Project/Dataset/filter_user_id.csv')
df_grouped = df_grouped[df_grouped['total_hours'] > 1]
#df_grouped['binrange'] = df_grouped.groupby('user_id').total_hours.apply(pd.cut, bins=5)
print(df_grouped.groupby('user_id').count())
#df_grouped1000['binbin'] = df_grouped1000.groupby('user_id').total_hours.apply(pd.qcut, q=[0, .2, .4, .6, .8, 1], duplicates = 'drop', labels=False)
#print(df_grouped1000['binbin'].value_counts())
#df_grouped1000.to_csv('/content/drive/My Drive/Web Mining Project/Dataset/user_binned_qcuttest.csv', index=False)

                   Unnamed: 0  Unnamed: 0.1  ...  count  bin
user_id                                      ...            
76561197960267615          17            17  ...     17   17
76561197960273775           9             9  ...      9    9
76561197960275370          10            10  ...     10   10
76561197960277861          45            45  ...     45   45
76561197960281570          44            44  ...     44   44
...                       ...           ...  ...    ...  ...
76561198431916153          10            10  ...     10   10
76561198445205155           9             9  ...      9    9
76561198445401211           7             7  ...      7    7
76561198452571798          11            11  ...     11   11
76561198726325843           8             8  ...      8    8

[25428 rows x 13 columns]


# 4. Join the Bin Ouput with our Final-Output

In [0]:
import pandas as pd
final_file = pd.read_csv("/content/drive/My Drive/Web Mining Project/Dataset/steam_games.csv")
final_file = final_file.drop_duplicates(subset=["id"], keep='first')
bins = pd.read_csv("/content/drive/My Drive/Web Mining Project/Dataset/user_binned_with_9_bins.csv")
final_file.rename(columns={'id':'product_id'}, inplace=True)
#bins = bins[['user_id', 'product_id', 'total_hours', 'bin']].copy()
merge_final_bin = final_file.merge(bins, how='inner', on='product_id')
#print(list(final_file))
#print(bins.head(5))
#print(merge_final_bin.head(5))
#merge_final_bin.to_csv("/content/drive/My Drive/Web Mining Project/Dataset/final_with_bins.csv", index = False)

merge_final_bin.drop(['early_access', 'price','release_date', 'app_name', 'title_y','url', 'discount_price', 'reviews_url', 'metascore', 'developer' ], axis=1, inplace=True)
merge_final_bin.head()
merge_final_bin.to_csv("/content/drive/My Drive/Web Mining Project/Dataset/final_with_bins.csv", index = False)



# Resampling


In [0]:
from sklearn.utils import resample



df_majority = train[train.bin==1.0]
df_majority_downsampled = resample(df_majority, 
                                replace=False,    # sample without replacement
                                n_samples=38858,     # to match minority class
                                random_state=123)
df_minority1 = train[train.bin==1.5]
df_minority1 = resample(df_minority1, 
                                 replace=True,    # sample without replacement
                                 n_samples=200000,     # to match minority class
                                 random_state=123)
df_minority2 = train[train.bin==2.0]
df_minority2 = resample(df_minority2, 
                                 replace=True,    # sample without replacement
                                 n_samples=195000,     # to match minority class
                                 random_state=123)
df_minority3 = train[train.bin==2.5]
df_minority3 = resample(df_minority3, 
                                 replace=True,    # sample without replacement
                                 n_samples=190000,     # to match minority class
                                 random_state=123)
df_minority4 = train[train.bin==3.0]
df_minority4 = resample(df_minority4, 
                                 replace=True,    # sample without replacement
                                 n_samples=185000,     # to match minority class
                                 random_state=123)
df_minority5 = train[train.bin==3.5]
df_minority5 = resample(df_minority5, 
                                 replace=True,    # sample without replacement
                                 n_samples=180000,     # to match minority class
                                 random_state=123)
df_minority6 = train[train.bin==4.0]
df_minority6 = resample(df_minority6, 
                                 replace=True,    # sample without replacement
                                 n_samples=175000,     # to match minority class
                                 random_state=123)
df_minority7 = train[train.bin==4.5]
df_minority7 = resample(df_minority7, 
                                 replace=True,    # sample without replacement
                                 n_samples=170000,     # to match minority class
                                 random_state=123)
df_minority8 = train[train.bin==5.0]
df_minority8 = resample(df_minority8, 
                                 replace=True,    # sample without replacement
                                 n_samples=220000,     # to match minority class
                                 random_state=123)

trainset = pd.concat([df_majority, df_minority1, df_minority2, df_minority3, df_minority4, df_minority5, df_minority6, df_minority7, df_minority8])

# 5. Creating Train and Test set

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [0]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/Web Mining Project/Dataset/"
%cd /content/drive/My Drive/Web Mining Project/Dataset/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 Archiv			     steam_games.csv
 cb5000.csv		     steam_games.json
 content_based_result.csv    steam_reviews.csv
'Data for Algorithms'	     steam_reviews.json
 filter_for_algorithms.csv   user_binned_qcut.csv
 filter_user_id.csv	     user_binned_qcuttest.csv
 final_with_bins.csv	     user_binned_with_9_bins.csv
 predicitons.csv	     user_binned_with_9_bins_new.csv
 predicitons_item.csv	     user_game_totalhours.csv
 predicitons_user.csv	     user_text.csv
/content/drive/My Drive/Web Mining Project/Dataset


In [0]:
ratings = pd.read_csv("content/drive/My Drive/Web Mining Project/Dataset/final_with_bins.csv")

In [0]:
#Filtering out User that have 
ratings = ratings.groupby('product_id').filter(lambda x: len(x) >= 5)
ratings = ratings.groupby('user_id').filter(lambda x: len(x) >= 15)

# Make correct types
ratings.product_id = ratings.product_id.astype('category')
ratings.head()
# Create user & product_id mappings.
userIds = ratings['user_id'].unique()
print("There are %i unique users.", len(userIds))
productIds = ratings['product_id'].unique()
print("There are %i unique products.", len(productIds))



There are %i unique users. 11243
There are %i unique products. 6649


In [0]:

#ratings.to_csv("/content/drive/My Drive/Web Mining Project/Dataset/filter_for_algorithms.csv", index = False)
ratings = pd.read_csv("/content/drive/My Drive/Web Mining Project/Dataset/filter_for_algorithms.csv")

ratings.shape

(308334, 14)

In [0]:
ratings['split'] = np.random.randn(ratings.shape[0], 1)
msk = np.random.rand(len(ratings)) <= 0.8
train = ratings[msk]
test = ratings[~msk]

In [0]:
!pip install scikit-surprise
from surprise import Dataset, Reader, SVD, accuracy, NMF
from surprise.model_selection import train_test_split 

reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(train[['user_id', 'product_id', 'bin']], reader)
trainset = data.build_full_trainset()


testset = [tuple(x) for x in test[['user_id', 'product_id', 'bin']].values]

     |████████████████████████████████| 6.5MB 27kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1675385 sha256=4ae2e3701e494bdcfdedf62f1ecbe3cc8f292e6d10dceedeb8d7e2b12cf00de0
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


## 5.1. Functions


In [0]:
def opt_parameters(exp_function, trainset, testset, config=None, param_space_iter=None):
    runs_info = collections.OrderedDict()
    for params in param_space_iter:
        # get key/id
        param_id = ';'.join(['%s=%s' % (key, value) for (key, value) in params.items()])
        print(param_id)
        # run experiment
        exp_info = run_exp(exp_function, trainset, testset, config, config_updates=params)
        # add results to OrderedDict
        runs_info[param_id] = exp_info
    
    return runs_info

In [0]:
def run_exp(exp_function, trainset, testset, config=None, config_updates=None):  
    if config is None:
        config = {}
    else:
        config = config.copy()
    
    if config_updates is not None:
        config.update(config_updates)
        
    return exp_function(config, trainset, testset)

In [0]:
def evaluation(predictions):
    """
        Get evaluation metrics (RMSE, MAE, MAP, NDCG) for predictions
    """
    res = {}
    
    res['rmse'] = accuracy.rmse(predictions)
    res['mae'] = accuracy.mae(predictions)
    res['ndcg'] = ndcg(predictions)
    res['map'] = mean_ap(predictions)
    
    return res

Evaluation Functions

In [0]:
from collections import defaultdict
from math import log2

import numpy as np


def mean_ap(predictions, verbose=True, k=10):
    predictions_u = defaultdict(list)
    true_u = defaultdict(list)

    for u0, i0, r0, est, _ in predictions:
        true_u[u0].append((r0, i0))
        predictions_u[u0].append((est, i0))
        
    #for index, (u0, i0, r0, est) in predictions.iterrows():
     #  true_u[u0].append((r0, i0))
      # predictions_u[u0].append((est, i0))
       
    aps = list()

    for u0 in predictions_u:
        if len(predictions_u[u0]) >= k:
            p = sorted(predictions_u[u0], reverse=True)[:k]
            p = [v for _,v in p]
            t = sorted(true_u[u0], reverse=True)[:k]
            t = [v for _,v in t]

            ap = 0
            correct = 0
            for i in range(len(p)):
                if p[i] in t:
                    correct += 1
                    ap += correct / (i+1)

            ap /= len(t)
            aps.append(ap)
            
    map_ = np.array(aps).mean()
    
    if verbose:
        print('MAP: {0:1.4f}'.format(map_))
    
    return map_


def ndcg(predictions, verbose=True, k=10):
    predictions_u = defaultdict(list)
    true_u = defaultdict(list)

    for u0, i0, r0, est, _ in predictions:
        true_u[u0].append((r0, i0))
        predictions_u[u0].append((est, i0))

    #for index,(u0, i0, r0, est) in predictions.iterrows():
     #   true_u[u0].append((r0, i0))
      #  predictions_u[u0].append((est, i0))

    idcg = np.arange(k+1)
    idcg[0:2] = 2
    idcg = 1 / np.log2(idcg)
    idcg = np.cumsum(idcg)

    ndcgs = list()

    for u0 in predictions_u:
        if len(predictions_u[u0]) >= k:
            p = sorted(predictions_u[u0], reverse=True)[:k]
            p = [v for _,v in p]
            t = sorted(true_u[u0], reverse=True)[:k]
            t = [v for _,v in t]

            dcg = 0
            correct = 0
            for i in range(len(p)):
                if p[i] in t:
                    correct += 1
                    if i == 0:
                        dcg += 1
                    else:
                        dcg += 1/log2(i+1)

            ndcg = dcg / idcg[correct]
            ndcgs.append(ndcg)

    ndcg_ = np.array(ndcgs).mean()
    
    if verbose:
        print('NDCG: {0:1.4f}'.format(ndcg_))
        
    return ndcg_

## 5.2. Baseline

In [0]:
import collections
from sklearn.model_selection import ParameterGrid
from surprise import BaselineOnly

In [0]:
def train_BaselineOnly(config, trainset, testset):
    predictions = BaselineOnly(
        bsl_options = {
            'method': config['method'],
            #'reg': config['reg']
        }).fit(trainset).test(testset)
    
    info = evaluation(predictions)
    
    # return metrics
    return info

In [0]:
config = {
    'method': 'als',
    #'reg': 0.02
}

param_grid = {
    'method': np.array(['als']),
    #'reg': np.array([0.1, 0.02, 0.03])
}

baselineOnly_runs_info = opt_parameters(train_BaselineOnly, config=config, trainset=trainset, testset=testset, param_space_iter=ParameterGrid(param_grid))

method=als
Estimating biases using als...
RMSE: 0.8500
MAE:  0.5220
NDCG: 0.7393
MAP: 0.6893


## 5.3. SVD

In [0]:
from sklearn.model_selection import ParameterGrid
import collections


In [0]:
def train_SVD(config, trainset, testset):
    predictions = SVD(
        n_epochs = config['epochs'],
        lr_all = config['learning_rate'],
        reg_all = config['regularization']
    ).fit(trainset).test(testset)
    
    info = evaluation(predictions)
    
   
    return info

In [0]:
svd_runs_info = opt_parameters(train_SVD, config={
    'epochs': 20,
    'learning_rate': 0.005,
    'regularization': 0.02
}, trainset=trainset, testset=testset, param_space_iter=ParameterGrid(param_grid = {
    'epochs': np.array([20, 35, 50]), # number of iterations of SVD procedure
    'learning_rate': np.array([0.002, 0.005]), # learning rate, default is 0.005
    'regularization': np.array([0.02, 0.05]), # regularization term for all parameters
}))

epochs=20;learning_rate=0.002;regularization=0.02
RMSE: 0.8608
MAE:  0.5334
NDCG: 0.7385
MAP: 0.6907
epochs=20;learning_rate=0.002;regularization=0.05
RMSE: 0.8590
MAE:  0.5326
NDCG: 0.7390
MAP: 0.6904
epochs=20;learning_rate=0.005;regularization=0.02
RMSE: 0.8611
MAE:  0.5188
NDCG: 0.7396
MAP: 0.6942
epochs=20;learning_rate=0.005;regularization=0.05
RMSE: 0.8545
MAE:  0.5165
NDCG: 0.7388
MAP: 0.6929
epochs=35;learning_rate=0.002;regularization=0.02
RMSE: 0.8577
MAE:  0.5227
NDCG: 0.7393
MAP: 0.6945
epochs=35;learning_rate=0.002;regularization=0.05
RMSE: 0.8541
MAE:  0.5197
NDCG: 0.7390
MAP: 0.6917
epochs=35;learning_rate=0.005;regularization=0.02
RMSE: 0.8682
MAE:  0.5182
NDCG: 0.7384
MAP: 0.6933
epochs=35;learning_rate=0.005;regularization=0.05
RMSE: 0.8570
MAE:  0.5120
NDCG: 0.7391
MAP: 0.6942
epochs=50;learning_rate=0.002;regularization=0.02
RMSE: 0.8612
MAE:  0.5183
NDCG: 0.7381
MAP: 0.6927
epochs=50;learning_rate=0.002;regularization=0.05
RMSE: 0.8536
MAE:  0.5154
NDCG: 0.7410
MA

## 5.4. NMF

In [0]:
from sklearn.model_selection import ParameterGrid
import collections

In [0]:
def train_NMF(config, trainset, testset):
    predictions = SVD(
        n_epochs = config['epochs'],
        lr_all = config['learning_rate'],
        reg_all = config['regularization']
    ).fit(trainset).test(testset)
    
    info = evaluation(predictions)
    
    # return metrics
    return info

In [0]:
svd_runs_info = opt_parameters(train_NMF, config={
    'n_factors': 5,
    'epochs': 20,
    'learning_rate': 0.005,
    'regularization': 0.02
}, trainset=trainset, testset=testset, param_space_iter=ParameterGrid(param_grid = {
    'n_factors': np.array([5, 10]),
    'epochs': np.array([20, 35, 50]), # number of iterations of NMF procedure
    'learning_rate': np.array([0.002, 0.005]), # learning rate, default is 0.005
    'regularization': np.array([0.02, 0.05]), # regularization term for all parameters
}))

epochs=20;learning_rate=0.002;n_factors=5;regularization=0.02
RMSE: 0.8605
MAE:  0.5329
NDCG: 0.7382
MAP: 0.6884
epochs=20;learning_rate=0.002;n_factors=5;regularization=0.05
RMSE: 0.8594
MAE:  0.5327
NDCG: 0.7384
MAP: 0.6879
epochs=20;learning_rate=0.002;n_factors=10;regularization=0.02
RMSE: 0.8600
MAE:  0.5326
NDCG: 0.7377
MAP: 0.6869
epochs=20;learning_rate=0.002;n_factors=10;regularization=0.05
RMSE: 0.8591
MAE:  0.5323
NDCG: 0.7395
MAP: 0.6875
epochs=20;learning_rate=0.005;n_factors=5;regularization=0.02
RMSE: 0.8617
MAE:  0.5194
NDCG: 0.7383
MAP: 0.6949
epochs=20;learning_rate=0.005;n_factors=5;regularization=0.05
RMSE: 0.8541
MAE:  0.5164
NDCG: 0.7402
MAP: 0.6947
epochs=20;learning_rate=0.005;n_factors=10;regularization=0.02
RMSE: 0.8610
MAE:  0.5192
NDCG: 0.7394
MAP: 0.6967
epochs=20;learning_rate=0.005;n_factors=10;regularization=0.05
RMSE: 0.8539
MAE:  0.5168
NDCG: 0.7402
MAP: 0.6950
epochs=35;learning_rate=0.002;n_factors=5;regularization=0.02
RMSE: 0.8567
MAE:  0.5216
NDCG

## 5.5. KNN

In [0]:
from surprise import KNNBasic
import pandas as pd

def train_KNNBasic(config, trainset, testset):
    predictions = KNNBasic(
        k = config['k'],
        sim_options = {
            'name': config['sim_name'],
            'min_support': config['min_support'],
            'user_based': config['user_based']
        }
    ).fit(trainset).test(testset)
    
    info = evaluation(predictions)
    #predictions = pd.DataFrame(predictions)
    print(predictions[0:1000])
    #predictions.to_csv("/content/drive/My Drive/Web Mining Project/Dataset/predicitons_user.csv")
    # return metrics
    return info

In [0]:
from surprise import KNNWithMeans
def train_KNNWithMeans(config, trainset, testset):
    predictions = KNNWithMeans(
        k = config['k'],
        sim_options = {
            'name': config['sim_name'],
            'min_support': config['min_support'],
            'user_based': config['user_based']
        }
    ).fit(trainset).test(testset)
    
    info = evaluation(predictions)
    
    # return metrics
    return info

### 5.5.1. Item-Based CF

Knn Baseline

In [0]:
config = {
    'k': 5,
    'sim_name': 'cosine',
    'min_support': 1,
    'user_based': False
}

param_grid = {
    'k': np.array([5, 10, 20]),
    'sim_name': np.array(['cosine', 'msd', 'pearson', 'pearson_baseline']),
    'min_support': np.array([1, 3, 5])
}


ib_knnbasic_runs_info = opt_parameters(train_KNNBasic, config=config, trainset=trainset, testset=testset, param_space_iter=ParameterGrid(param_grid), )

Output hidden; open in https://colab.research.google.com to view.

KNN with Means

In [0]:
config = {
    'k': 40,
    'sim_name': 'cosine',
    'min_support': 1,
    'user_based': False
}

param_grid = {
    'k': np.array([30, 40, 50, 60]),
    'sim_name': np.array(['cosine', 'msd', 'pearson_baseline']),
    'min_support': np.array([1, 3, 5])
}


ib_knnmeans_runs_info = opt_parameters(train_KNNWithMeans, config=config, trainset=trainset, testset=testset, param_space_iter=ParameterGrid(param_grid))

k=30;min_support=1;sim_name=cosine
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.8770
MAE:  0.5252
NDCG: 0.7372
MAP: 0.6892
k=30;min_support=1;sim_name=msd
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8740
MAE:  0.5152
NDCG: 0.7374
MAP: 0.6883
k=30;min_support=1;sim_name=pearson_baseline
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9409
MAE:  0.5340
NDCG: 0.7428
MAP: 0.7051
k=30;min_support=3;sim_name=cosine
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9182
MAE:  0.5435
NDCG: 0.7404
MAP: 0.6991
k=30;min_support=3;sim_name=msd
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9142
MAE:  0.5299
NDCG: 0.7392
MAP: 0.6945
k=30;min_support=3;sim_name=pearson_baseline
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similari

### 5.5.2. User-Based CF

KNN Baseline

In [0]:
config = {
    'k': 5,
    'sim_name': 'cosine',
    'min_support': 1,
    'user_based': True
}

param_grid = {
    'k': np.array([5, 10, 20]),
    'sim_name': np.array(['cosine', 'msd', 'pearson', 'pearson_baseline']),
    'min_support': np.array([1, 3, 5])
}


ib_knnbasic_runs_info = opt_parameters(train_KNNBasic, config=config, trainset=trainset, testset=testset, param_space_iter=ParameterGrid(param_grid))

Output hidden; open in https://colab.research.google.com to view.

KNN with means

In [0]:
config = {
    'k': 40,
    'sim_name': 'cosine',
    'min_support': 1,
    'user_based': True
}

param_grid = {
    'k': np.array([30, 40, 50, 60]),
    'sim_name': np.array(['cosine', 'msd', 'pearson_baseline']),
    'min_support': np.array([1, 3, 5])
}


ib_knnmeans_runs_info = opt_parameters(train_KNNWithMeans, config=config, trainset=trainset, testset=testset, param_space_iter=ParameterGrid(param_grid))

k=30;min_support=1;sim_name=cosine
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.8839
MAE:  0.5310
NDCG: 0.7447
MAP: 0.7190
k=30;min_support=1;sim_name=msd
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8794
MAE:  0.5292
NDCG: 0.7449
MAP: 0.7175
k=30;min_support=1;sim_name=pearson_baseline
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9054
MAE:  0.5462
NDCG: 0.7430
MAP: 0.7010
k=30;min_support=3;sim_name=cosine
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9301
MAE:  0.5675
NDCG: 0.7425
MAP: 0.6992
k=30;min_support=3;sim_name=msd
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9328
MAE:  0.5686
NDCG: 0.7430
MAP: 0.7000
k=30;min_support=3;sim_name=pearson_baseline
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similari

## 5.6. Hybrid

In [0]:
from surprise import KNNBasic
import pandas as pd

def train_KNNBasic(config, trainset, testset):
    predictions = KNNBasic(
        k = config['k'],
        sim_options = {
            'name': config['sim_name'],
            'min_support': config['min_support'],
            'user_based': config['user_based']
        }
    ).fit(trainset).test(testset)
    
    info = evaluation(predictions)
    predictions = pd.DataFrame(predictions)
    #print(predictions[0:1000])
    predictions.to_csv("/content/drive/My Drive/Web Mining Project/Dataset/predicitons_user.csv")
   
    return info

In [0]:

config = {
    'k': 20, #for item based
    #'k' : 10, #for user based
    'sim_name': 'pearson',
    'min_support': 5,
    'user_based': False #item-based
    #'user_based': True #user-based
}

param_grid = {}
ib_knnbasic_runs_info = opt_parameters(train_KNNBasic, config=config, trainset=trainset, testset=testset, param_space_iter=ParameterGrid(param_grid), )


Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 1.0931
MAE:  0.6751
NDCG: 0.7608
MAP: 0.7364


In [0]:

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

def evaluation(predictions):
    """
        Get evaluation metrics (RMSE, MAE, MAP, NDCG) for predictions
    """
    res = {}
    
    #res['rmse'] = accuracy.rmse(predictions)
    y_true = test1['r_ui']
    y_pred = test1['est']
    res['rmse'] = mean_squared_error(y_true, y_pred, squared=False)
    
    res['mae'] = mean_absolute_error(y_true, y_pred)

    res['ndcg'] = ndcg(predictions)
    res['map'] = mean_ap(predictions)
    
    return res

In [0]:
test1 = pd.read_csv("/content/drive/My Drive/Web Mining Project/Dataset/predicitons_user.csv")
test1 = test1[['uid', 'r_ui','est', 'iid']].copy()
print(test1.head())

test2 = pd.read_csv("/content/drive/My Drive/Web Mining Project/Dataset/predicitons_item.csv")
test2 = test2[['uid', 'r_ui','est', 'iid']].copy()
print(test2.head())

#merge = test1.merge(test2, how='inner', on='uid')
test1['est2']= test2['est']
print(test1.head())
print(test1.dtypes)

test1['hybrid'] = (test1.est + test1.est2)/2
print(test1.head())
test1 = test1.drop(columns = ['est', 'est2'], axis = 1)
print(test1.head())
test1 = test1.rename({'hybrid': 'est'}, axis=1)
print(test1.head())

print(evaluation(test1))


                 uid  r_ui       est       iid
0  76561198032352680   1.0  1.467977  282010.0
1  76561197964108437   1.0  1.000000      70.0
2  76561197967634305   1.0  1.132868      70.0
3  76561197970413755   1.0  1.000000      70.0
4  76561197972054287   2.0  1.467977      70.0
                 uid  r_ui       est       iid
0  76561198032352680   1.0  1.467977  282010.0
1  76561197964108437   1.0  1.474768      70.0
2  76561197967634305   1.0  1.845441      70.0
3  76561197970413755   1.0  1.058530      70.0
4  76561197972054287   2.0  2.388059      70.0
                 uid  r_ui       est       iid      est2
0  76561198032352680   1.0  1.467977  282010.0  1.467977
1  76561197964108437   1.0  1.000000      70.0  1.474768
2  76561197967634305   1.0  1.132868      70.0  1.845441
3  76561197970413755   1.0  1.000000      70.0  1.058530
4  76561197972054287   2.0  1.467977      70.0  2.388059
uid       int64
r_ui    float64
est     float64
iid     float64
est2    float64
dtype: object


## 5.7. Naïve Bayes


In [0]:


df = ratings[['user_id', 'product_id', 'bin']].copy()
df_unique_1000 = df['user_id'].unique()[0:1000]
df_nb = df[df['user_id'].isin(df_unique_1000)]

print(df_nb.shape)
print(df.shape)

(32540, 3)
(475665, 3)


In [0]:
# naive bayes for recommendation system

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split




df_nb['product_id'] = df_nb['product_id'].astype(int)
df_nb['bin'] = df_nb['bin'].astype(str)
df_matrix = df_nb.pivot(values='bin', index='user_id', columns='product_id')

# print(df_matrix.head(10))
# print(df['user_id'].nunique())
# print(df['product_id'].nunique())
print("Matrix shape: " + str(df_matrix.shape))

count=0

# array storing all recommendations
arr_rec = []

for user in df_matrix.index.tolist():
	print(count)
	
	# prods user has not rated
	prods_unrated = df_matrix.columns[df_matrix.loc[user].isnull()]
	prods_unrated = prods_unrated.tolist()
	#print(prods_unrated)
	
	# prods user has rated
	prods_rated = set(list(df_matrix)) - set(prods_unrated)
	prods_rated = list(prods_rated)
	#print(prods_rated)

	# predictions for all products for current user
	p_user = []

	if len(prods_unrated) == 0:
		# user rated all products, no need to recommend
		continue

	for p in prods_unrated:
		# probability of different ratings for p
		p_pr = []

		for r in range(1,5):
			p_Y = len(df_matrix[df_matrix[p]==str(r)]) / (len(df_matrix) - 1)
			p_XY = 1

			# users who give rating r to unrated item p
			df_matrix_Y = df_matrix[df_matrix[p]==str(r)].copy()

			# if nobody rated p before
			if len(df_matrix_Y) == 0:
				p_pr.append(0.0)
				continue
			
			# print("p unrated: " + str(p))
			# print("r: " + str(r))
			# print(df_matrix_Y.head(10))

			for q in prods_rated:
				
				# print("q: " + str(q))

				# rating of user on item q
				user_r_q = df_matrix.at[user, q]

				# users who give the same rating to item q as user
				df_matrix_r_q = df_matrix_Y[df_matrix[q]==str(user_r_q)]
				
				p_XY = p_XY * float(len(df_matrix_r_q) + 1.0) / float(len(df_matrix_Y) + 5.0)

				# print(str(len(df_matrix_r_q)) + "/" + str(len(df_matrix_Y)))
				# print("p_XY: " + str(p_XY))
				# print("p_Y: " + str(p_Y))

			p_YX = p_XY * p_Y
			p_pr.append(p_YX)
			# print("P(item " + str(p) + "=" + str(r) + "|X) = " + str(p_YX))	

		#print(p_pr)
		#print(str(user) + " will most likely give item " + str(p) + " a score of " + str(max(p_pr)))

		p_user.append(max(p_pr))

	#rec_dict = dict(zip(prods_unrated, p_user))
	#rec_dict_sorted = {k: v for k, v in sorted(rec_dict.items(), key=lambda item: item[1], reverse=True)}
	# print(p_user)
	# print(rec_dict_sorted)
	#print("top recommendation for " + str(user) + ": " + str(prods_unrated[p_user.index(max(p_user))]))
	#print('\n\n')
	for i in range(0,len(p_user)):
		arr_rec.append([user,prods_unrated[i],p_user[i]])
	df_rec = pd.DataFrame(arr_rec, columns=['user_id','product_id','rating'])

	count = count+1
	# if count > 100:
	# 	break


df_rec.to_csv("nb_result.csv", index=False)


In [0]:
#Test1
ratings.head()

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import ParameterGrid
import collections
def convert(ratings):
    ids = []
    length = 10000
    users = ratings['user_id'].values
    product = ratings['product_id'].values
    for i in range(0, length):
        ids.append(str(users[i])+' '+str(product[i]))
    return ids
def predict(train_data, test_data):
    vectorizer = CountVectorizer()
    train_ids = []
    #length = len(train_data['user_id'])
    train_ids = convert(train_data)
    inputs = vectorizer.fit_transform(train_ids)
    classifier = MultinomialNB()
    outputs = train_data['bin'].values
    classifier.fit(inputs, outputs)
    test_ids = convert(test_data)
    test = vectorizer.transform(test_ids)
    predictions = classifier.predict(test)
    
    return predictions

    #convert(trainset)
    #convert(testset)
predictions = predict(trainset, testset)
info = evaluation(predictions)
print(info)

In [0]:
def trainnb(config, trainset, testset):
    predictions = MultinomialNB().fit(trainset).test(testset)
    
    info = evaluation(predictions)
    
    # return metrics
    return info

In [0]:
svd_runs_info = opt_parameters(trainnb, convert, predict, config={}, trainset=trainset, testset=testset, param_space_iter=ParameterGrid(param_grid = {}))

## 5.8. Cross Validation

In [0]:
from sklearn.model_selection import train_test_split
from surprise.model_selection import cross_validate
from surprise import KNNBasic
#reader = Reader(rating_scale=(1, 5))
#trainset, testset = train_test_split(ratings[['user_id','product_id','bin']], test_size=0.2, random_state=0)
#trainset.index = range(len(trainset)) # 75627
#testset.index = range(len(testset)) # 25209


trainset = Dataset.load_from_df(trainset, reader)

#testset = testset.values.tolist()
benchmark = []
i = 0
alg_list = ['SVD(n_factors=50, n_epochs=10, lr_all=0.002)', 'KNNBasic(k=40, sim_options = cosine, user_based=False)']
for algorithm in [SVD(n_factors=50, n_epochs=10, lr_all=0.002), 
                  KNNBasic(k=40, sim_options={'name': 'cosine','user_based': False}, 
                  KNNBasic(k=40, sim_options={'name': 'cosine','user_based': True})]:
    # Perform cross validation
    results = cross_validate(algorithm, trainset, measures=['RMSE','MAE'], cv=5, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([alg_list[i]], index=['Algorithm']))
    benchmark.append(tmp)
    i+=1
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')

In [0]:
from surprise.model_selection import KFold
kf = KFold(n_splits=5)

#algo = SVD()
config={
    'epochs': 20,
    'learning_rate': 0.005,
    'regularization': 0.02
}
for trainset, testset in kf.split(data):
  
  #train_SVD(config, trainset, testset)
  svd_runs_info = opt_parameters(train_SVD, config={
      'epochs': 20,
    'learning_rate': 0.005,
    'regularization': 0.02}, trainset=trainset, testset=testset, param_space_iter=ParameterGrid(param_grid = {
    'epochs': np.array([20, 35]), # number of iterations of SVD procedure
    'learning_rate': np.array([0.002]), # learning rate, default is 0.005
    'regularization': np.array([0.02, 0.05]), # regularization term for all parameters
  }))

    # train and test algorithm.
    #algo.fit(trainset)
    #predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    #accuracy.rmse(predictions, verbose=True)

epochs=20;learning_rate=0.002;regularization=0.02
NDCG: 0.7399
MAP: 0.6998
epochs=20;learning_rate=0.002;regularization=0.05
NDCG: 0.7405
MAP: 0.7010
epochs=35;learning_rate=0.002;regularization=0.02
NDCG: 0.7421
MAP: 0.7086
epochs=35;learning_rate=0.002;regularization=0.05
NDCG: 0.7417
MAP: 0.7050
epochs=20;learning_rate=0.002;regularization=0.02
NDCG: 0.7376
MAP: 0.6955
epochs=20;learning_rate=0.002;regularization=0.05
NDCG: 0.7386
MAP: 0.6967
epochs=35;learning_rate=0.002;regularization=0.02
NDCG: 0.7386
MAP: 0.6991
epochs=35;learning_rate=0.002;regularization=0.05
NDCG: 0.7392
MAP: 0.7026
epochs=20;learning_rate=0.002;regularization=0.02
NDCG: 0.7417
MAP: 0.6979
epochs=20;learning_rate=0.002;regularization=0.05
NDCG: 0.7422
MAP: 0.6928
epochs=35;learning_rate=0.002;regularization=0.02
NDCG: 0.7414
MAP: 0.7003
epochs=35;learning_rate=0.002;regularization=0.05
NDCG: 0.7428
MAP: 0.7019
epochs=20;learning_rate=0.002;regularization=0.02
NDCG: 0.7403
MAP: 0.6975
epochs=20;learning_rate=0